# Emerging Technologies Assessment

## Introduction

This notebook explores the difference between classical and quantum algorithms through the
Deutsch and Deutsch–Jozsa problems. These problems demonstrate how quantum computation can
provide exponential speedups over classical deterministic algorithms for certain tasks.

In particular, the task is to determine whether a Boolean function is constant or balanced.
This may require evaluating all possible inputs while quantum algorithms can solve the problem with a single oracle query.

The notebook is structured into five problems, each building on the previous to progress
from classical computation to quantum implementations using Qiskit.

## Problem 1: Generating Random Boolean Functions

A Boolean function with four inputs has 2⁴ = 16 possible input combinations.
According to the Deutsch–Jozsa promise, the function is guaranteed to be either:

- Constant: returns the same output for all inputs, or  
- Balanced: returns True for exactly half of the inputs and False for the other half.

In this problem, I construct a Python function that randomly generates either a constant
or balanced Boolean function. Balanced functions are created by assigning True to half of
the input combinations and False to the remaining half, ensuring the promise condition is met.


In [11]:
# Code Cell

## Problem 2: Classical Testing for Function Type

In [12]:
# Code cell

## Problem 3: Quantum Oracles

In [13]:
# Code cell

## Problem 4: Deutsch's Algorithm with Qiskit


In [14]:
# Code cell

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm

In [15]:
# Code cell